In [5]:
import numpy as np
import pandas as pd
import xgboost as xgb
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from flaml import BlendSearch
import os




In [9]:
X_train = pd.read_csv("/home/jovyan/work/juningpar/onsp/xgb_june/data/X_train.csv")
y_train  = pd.read_csv("/home/jovyan/work/juningpar/onsp/xgb_june/data/y_train.csv")
X_test = pd.read_csv("/home/jovyan/work/juningpar/onsp/xgb_june/data/X_test.csv")
y_test = pd.read_csv("/home/jovyan/work/juningpar/onsp/xgb_june/data/y_test.csv")

In [11]:

# Configuration
n_fold = 5
random_seed = 42
threshold = 0.8  # ปรับ threshold เป็น 0.8
patience = 5
max_trials = 1500

target_dir = './model2/'
os.makedirs(target_dir, exist_ok=True)

def objective(config):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'max_depth': config['max_depth'],
        'min_child_weight': config['min_child_weight'],
        'subsample': config['subsample'],
        'learning_rate': config['learning_rate'],
        'colsample_bylevel': config['colsample_bylevel'],
        'colsample_bytree': config['colsample_bytree'],
        'gamma': config['gamma'],
        'reg_lambda': config['reg_lambda'],
        'reg_alpha': config['reg_alpha'],
        'n_estimators': config['n_estimators']
    }
    
    skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=random_seed)
    val_scores, acc_scores, f1_scores, precision_scores, f1_macro_scores = [], [], [], [], []
    results_list = []
    
    for fold, (train_index, test_index) in tqdm(enumerate(skf.split(X_train, y_train)), total=n_fold, desc=f"Evaluating Config", leave=True, colour='blue', dynamic_ncols=True):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        model = xgb.XGBClassifier(**params)
        model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)], verbose=False)
        preds_prob = model.predict_proba(X_val_fold)[:, 1]
        preds = (preds_prob >= threshold).astype(int)
        
        val_score = recall_score(y_val_fold, preds)
        acc_score = accuracy_score(y_val_fold, preds)
        precision = precision_score(y_val_fold, preds, zero_division=0)
        f1 = f1_score(y_val_fold, preds, zero_division=0)
        f1_macro = f1_score(y_val_fold, preds, average='macro', zero_division=0)
        
        val_scores.append(val_score)
        acc_scores.append(acc_score)
        precision_scores.append(precision)
        f1_scores.append(f1)
        f1_macro_scores.append(f1_macro)
        
        results_list.append([fold + 1, val_score, acc_score, precision, f1, f1_macro, params['n_estimators']])
    
    results_df = pd.DataFrame(results_list, columns=['Fold', 'Recall', 'Accuracy', 'Precision', 'F1', 'F1_macro', 'N_Estimators'])
    results_df.to_csv(os.path.join(target_dir, 'kf_results2.csv'), mode='a', header=not os.path.exists(os.path.join(target_dir, 'kf_results2.csv')), index=False)
    
    return {
        'recall': np.mean(val_scores),
        'accuracy': np.mean(acc_scores),
        'precision': np.mean(precision_scores),
        'f1': np.mean(f1_scores),
        'f1_macro': np.mean(f1_macro_scores)
    }

start_time = time.time()

search_space = {
    'max_depth': tune.randint(3, 10),
    'min_child_weight': tune.loguniform(1e-8, 1e5),
    'subsample': tune.uniform(0.5, 1.0),
    'learning_rate': tune.loguniform(1e-5, 1.0),
    'colsample_bylevel': tune.uniform(0.5, 1.0),
    'colsample_bytree': tune.uniform(0.5, 1.0),
    'gamma': tune.loguniform(1e-8, 1e2),
    'reg_lambda': tune.loguniform(1e-8, 1e2),
    'reg_alpha': tune.loguniform(1e-8, 1e2),
    'n_estimators': tune.randint(100, 500)
}

blend_search = BlendSearch()
analysis = tune.run(
    objective,
    config=search_space,
    metric='recall',
    mode='max',
    search_alg=blend_search,
    num_samples=max_trials,
    verbose=1
)

time_taken = time.time() - start_time
best_config = analysis.best_config
best_result = analysis.best_result
best_result['total_time'] = time_taken

# เทรนโมเดลที่ดีที่สุด
best_model = xgb.XGBClassifier(**best_config)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# บันทึกผลลัพธ์ลง CSV
best_params_df = pd.DataFrame([{**best_config, **best_result, 'time_taken': time_taken}])
best_params_df.to_csv(os.path.join(target_dir, 'best_xgb_params2.csv'), mode='a', header=not os.path.exists(os.path.join(target_dir, 'best_xgb_params2.csv')), index=False)

# วาด Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title(f'Confusion Matrix (Threshold = {threshold})')
plt.show()

# บันทึกเวลาในการรันทั้งหมด
with open(os.path.join(target_dir, 'execution_time.txt'), 'w') as f:
    f.write(f'Total execution time: {time_taken:.2f} seconds\n')


/opt/conda/lib/python3.12/site-packages/flaml/tune/searcher/suggestion.py:886: FutureWarning: IntUniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~optuna.distributions.IntDistribution` instead.
  return ot.distributions.IntUniformDistribution(
/opt/conda/lib/python3.12/site-packages/flaml/tune/searcher/suggestion.py:871: FutureWarning: LogUniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~optuna.distributions.FloatDistribution` instead.
  return ot.distributions.LogUniformDistribution(domain.lower, domain.upper)
/opt/conda/lib/python3.12/site-packages/flaml/tune/searcher/suggestion.py:876: FutureWarning: UniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~opt

[flaml.tune.tune: 03-10 02:57:32] {891} INFO - trial 1 config: {'max_depth': 7, 'min_child_weight': 2.59692290753025, 'subsample': 0.681404614239038, 'learning_rate': 0.0006971034809046617, 'colsample_bylevel': 0.877743449018298, 'colsample_bytree': 0.6331962102154012, 'gamma': 0.3648436787087464, 'reg_lambda': 1.3680931439390769e-08, 'reg_alpha': 0.4711576651883603, 'n_estimators': 138}


Evaluating Config:   0%|          | 0/5 [00:00<?, ?it/s]

[flaml.tune.tune: 03-10 02:57:33] {891} INFO - trial 2 config: {'max_depth': 8, 'min_child_weight': 1.861124137551614e-08, 'subsample': 0.8168241174631377, 'learning_rate': 0.05546505049361988, 'colsample_bylevel': 0.7492535061512953, 'colsample_bytree': 0.6123983227654238, 'gamma': 9.563759526973396e-07, 'reg_lambda': 0.40300192627116094, 'reg_alpha': 4.910303832962881e-07, 'n_estimators': 135}


/opt/conda/lib/python3.12/site-packages/optuna/distributions.py:781: FutureWarning: IntUniformDistribution(high=9, low=3, step=1) is deprecated and internally converted to IntDistribution(high=9, log=False, low=3, step=1). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.12/site-packages/optuna/distributions.py:781: FutureWarning: LogUniformDistribution(high=100000.0, low=1e-08) is deprecated and internally converted to FloatDistribution(high=100000.0, log=True, low=1e-08, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.12/site-packages/optuna/distributions.py:781: FutureWarning: UniformDistribution(high=1.0, low=0.5) is deprecated and internally converted to FloatDistribution(high=1.0, log=False, low=0.5, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.12/site-packages/o

...

[flaml.tune.tune: 03-10 03:08:39] {891} INFO - trial 61 config: {'max_depth': 7, 'min_child_weight': 87349.90573154624, 'subsample': 0.8679600307902509, 'learning_rate': 0.16873099582610648, 'colsample_bylevel': 0.920856258408962, 'colsample_bytree': 0.9062069821617437, 'gamma': 24.091186375378214, 'reg_lambda': 2.349201049952294e-07, 'reg_alpha': 35.87858110026275, 'n_estimators': 338}


/opt/conda/lib/python3.12/site-packages/optuna/distributions.py:781: FutureWarning: IntUniformDistribution(high=9, low=3, step=1) is deprecated and internally converted to IntDistribution(high=9, log=False, low=3, step=1). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.12/site-packages/optuna/distributions.py:781: FutureWarning: LogUniformDistribution(high=100000.0, low=1e-08) is deprecated and internally converted to FloatDistribution(high=100000.0, log=True, low=1e-08, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.12/site-packages/optuna/distributions.py:781: FutureWarning: UniformDistribution(high=1.0, low=0.5) is deprecated and internally converted to FloatDistribution(high=1.0, log=False, low=0.5, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.12/site-packages/o

Evaluating Config:   0%|          | 0/5 [00:00<?, ?it/s]

Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x7f60ff5cfef0>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/xgboost/core.py", line 637, in _next_wrapper
    return self._handle_exception(lambda: self.next(input_data), 0)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/xgboost/core.py", line 550, in _handle_exception
    return fn()
           ^^^^
  File "/opt/conda/lib/python3.12/site-packages/xgboost/core.py", line 637, in <lambda>
    return self._handle_exception(lambda: self.next(input_data), 0)
                                          ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/xgboost/data.py", line 1402, in next
    input_data(**self.kwargs)
  File "/opt/conda/lib/python3.12/site-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)

XGBoostError: [03:08:53] /workspace/src/data/proxy_dmatrix.h:172: Unknown type: v
Stack trace:
  [bt] (0) /opt/conda/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x25c1ac) [0x7f610e85c1ac]
  [bt] (1) /opt/conda/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x5435ac) [0x7f610eb435ac]
  [bt] (2) /opt/conda/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x5483e5) [0x7f610eb483e5]
  [bt] (3) /opt/conda/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x54b99c) [0x7f610eb4b99c]
  [bt] (4) /opt/conda/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x4fb3ea) [0x7f610eafb3ea]
  [bt] (5) /opt/conda/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(XGQuantileDMatrixCreateFromCallback+0x18c) [0x7f610e773a5c]
  [bt] (6) /opt/conda/lib/python3.12/lib-dynload/../../libffi.so.8(+0x6d8a) [0x7f61815bfd8a]
  [bt] (7) /opt/conda/lib/python3.12/lib-dynload/../../libffi.so.8(+0x61cd) [0x7f61815bf1cd]
  [bt] (8) /opt/conda/lib/python3.12/lib-dynload/../../li

NameError: name 'xgboost' is not defined

In [ ]:
# plt.figure(figsize=(5, 4))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.title(f'Confusion Matrix (Threshold = {threshold})')
# plt.show()

In [ ]:
# best_config = analysis.best_config
# best_score = analysis.best_result['recall']
# best_accuracy = analysis.best_result['accuracy']
# best_f1 = analysis.best_result['f1']
# best_result = analysis.best_result

In [ ]:

# # บันทึกผลลัพธ์ลง CSV
# best_params_df = pd.DataFrame([{**best_config, **best_result, 'trial': trial}])
# best_params_df.to_csv('./model2/best_xgb_params2.csv', mode='a', header=not pd.io.common.file_exists('./model2/best_xgb_params2.csv'), index=False)

# # บันทึกค่าทั้งหมดของแต่ละ K-Fold ลง CSV
# results_df = pd.DataFrame(results_list, columns=['Trial', 'Fold', 'Recall', 'Accuracy', 'Precision', 'F1', 'F1_macro', 'Best_n_estimators', 'KF_Time'])
# results_df.to_csv('./model2/kf_results2.csv', mode='a', header=not pd.io.common.file_exists('./model2/kf_results2.csv'), index=False)
# results = pd.DataFrame([[str(best_config), best_score, best_accuracy, best_f1, time_taken, trial]], columns=['best_params', 'best_score', 'best_accuracy', 'best_f1', 'time_taken', 'trial'])
# results.to_csv('./time/xgb_best_params2.csv', mode='a', header=not pd.io.common.file_exists('./time/xgb_best_params2.csv'), index=False)

In [ ]:
# # เทรนโมเดลที่ดีที่สุดและบันทึกค่าการคาดการณ์ลง CSV
# best_model = xgb.XGBClassifier(**best_config)
# best_model.fit(X_train, y_train)
# y_pred = best_model.predict(X_test)

In [ ]:
# # บันทึกผลลัพธ์ลง CSV
# best_params_df = pd.DataFrame([{**best_config, **best_result, 'trial': trial}])
# best_params_df.to_csv('./model2/best_xgb_params2.csv', mode='a', header=not pd.io.common.file_exists('./model2/best_xgb_params2.csv'), index=False)

# # บันทึกค่าทั้งหมดของแต่ละ K-Fold ลง CSV
# results_df = pd.DataFrame(results_list, columns=['Trial', 'Fold', 'Recall', 'Accuracy', 'Precision', 'F1', 'F1_macro', 'Best_n_estimators', 'KF_Time'])
# results_df.to_csv('./model2/kf_results2.csv', mode='a', header=not pd.io.common.file_exists('./model2/kf_results2.csv'), index=False)

# time_taken = time.time() - start_time
# results = pd.DataFrame([[str(best_config), best_score, best_accuracy, best_f1, time_taken, trial]], columns=['best_params', 'best_score', 'best_accuracy', 'best_f1', 'time_taken', 'trial'])
# results.to_csv('./time/xgb_best_params2.csv', mode='a', header=not pd.io.common.file_exists('./time/xgb_best_params2.csv'), index=False)

# prediction_results = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred})
# prediction_results.to_csv('./predictions/best_model_predictions2.csv', index=False)
